# Install and import torch-geometric

In [1]:
import torch

CUDA_AVAILABLE = torch.cuda.is_available()

try:
    import torch_geometric

except ImportError:
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.', '') if CUDA_AVAILABLE else 'cpu'
    %pip install \
        torch-scatter==latest+{CUDA} \
        torch-sparse==latest+{CUDA} \
        torch-cluster==latest+{CUDA} \
        torch-spline-conv==latest+{CUDA} \
        torch-geometric \
        -f https://pytorch-geometric.com/whl/torch-{TORCH}.html

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import accuracy, add_self_loops, to_dense_adj
import torch_geometric.transforms as T

import os
import glob

device = torch.device('cuda' if CUDA_AVAILABLE else 'cpu')

# Load and preprocess data

In [2]:
# load Cora dataset
dataset = Planetoid(root='./datasets/Cora', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0].to(device)

In [3]:
# split dataset
train_node = data.train_mask
train_target = data.y[data.train_mask]
valid_node = data.val_mask
valid_target = data.y[data.val_mask]
test_node = data.test_mask
test_target = data.y[data.test_mask]

In [4]:
in_dim = dataset.num_node_features
in_dim

1433

In [5]:
class_cardinality = dataset.num_classes
class_cardinality

7

In [6]:
node_features = data.x
A = data.edge_index

In [7]:
len(data.train_mask)

2708

In [8]:
data.train_mask

tensor([ True,  True,  True,  ..., False, False, False], device='cuda:0')

In [9]:
len(data.y)

2708

In [10]:
data.y

tensor([3, 4, 4,  ..., 3, 3, 3], device='cuda:0')

In [11]:
A

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]], device='cuda:0')

In [12]:
A.shape

torch.Size([2, 10556])

In [13]:
to_dense_adj(A)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 1., 0.]]], device='cuda:0')

In [14]:
to_dense_adj(A).shape

torch.Size([1, 2708, 2708])

In [15]:
to_dense_adj(A).sum()

tensor(10556., device='cuda:0')

## Degree Normalization
$\hat{A} = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}$ (~: Self Loop)

In [16]:
A = to_dense_adj(A).squeeze(0)
self_loop = torch.eye(*A.shape).to(A)
A += self_loop  # add self-loops

deg = A.sum(1)
deg_inv = deg.pow(-0.5)
deg_inv.masked_fill_(deg_inv == torch.inf, 0.)
deg_inv = deg_inv * self_loop

A = deg_inv.mm(A).mm(deg_inv)
A

tensor([[0.2500, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2500, 0.2041,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2041, 0.1666,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.4999, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2000, 0.2000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2000, 0.2000]],
       device='cuda:0')

# Implement GCN

$\hat{A}XW$

In [17]:
class GCN(nn.Linear):

    def __init__(self, in_features, out_features):
        super().__init__(in_features, out_features, bias=True)
        # nn.init.xavier_uniform_(self.weight)

    def forward(self, x, adj):
        return torch.matmul(adj, super().forward(x))


$Z = f(X,A) = \text{softmax}(\hat{A} \text{ReLU} (\hat{A}XW^{(0)}) W^{(1)})$

In [18]:
class GCNNet(nn.Module):

    def __init__(self, in_dim, hidden_dim, class_cardinality, dropout_rate):
        super().__init__()
        self.gcn0 = GCN(in_dim, hidden_dim)
        self.gcn1 = GCN(hidden_dim, class_cardinality)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x, adj):
        x = self.dropout(x)
        x = self.gcn0(x, adj)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.gcn1(x, adj)
        return x.log_softmax(dim=1)


# Train

In [19]:
hidden_dim = 64
dropout_rate = 0.8
lr = 1e-2
weight_decay = 1e-3
epochs = 10000
patience = 200

In [20]:
loss_values = []
bad_counter = 0
best = epochs + 1
best_epoch = 0

model = GCNNet(in_dim, hidden_dim, class_cardinality, dropout_rate).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [21]:
def step(epoch):

    log = "[Epoch] {:0>3} ".format(epoch)

    model.train().zero_grad()

    log_logits = model(node_features, A)
    train_loss = F.nll_loss(log_logits[train_node], train_target)
    train_acc = accuracy(log_logits[train_node].argmax(1), train_target)

    train_loss.backward()
    optimizer.step()

    log += " [Train] Loss: {:6.4f}, Accuracy: {:6.4f}".format(train_loss, train_acc)

    model.eval()

    with torch.no_grad():
        log_logits = model(node_features, A)
        valid_loss = F.nll_loss(log_logits[valid_node], valid_target)
        valid_acc = accuracy(log_logits[valid_node].argmax(1), valid_target)

    log += " [Valid] Loss: {:6.4f}, Accuracy: {:6.4f}".format(valid_loss, valid_acc)
    print(log)

    return valid_loss.item()


@torch.no_grad()
def compute_test():

    model.eval()

    log_logits = model(node_features, A)
    test_loss = F.nll_loss(log_logits[valid_node], valid_target)
    test_acc = accuracy(log_logits[valid_node].argmax(1), valid_target)

    return test_loss.item(), test_acc

In [22]:
try:

    for e in range(epochs):
        loss_values.append(step(e))
        torch.save(model.state_dict(), "%s.pth" % e)
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = e
            bad_counter = 0
        else:
            bad_counter += 1
        if bad_counter == patience:
            break

    print("Optimization Finished!\nLoading %sth model\n" % best_epoch)
    model.load_state_dict(torch.load("%s.pth" % best_epoch))

    best_test_loss, best_test_acc = compute_test()
    print("[Test] Loss: {:6.4f}, Accuracy: {:6.4f}".format(best_test_loss, best_test_acc))

finally:
    for file in glob.iglob('*.pth'):
        os.remove(file)


[Epoch] 000  [Train] Loss: 1.9529, Accuracy: 0.1429 [Valid] Loss: 1.9515, Accuracy: 0.0580
[Epoch] 001  [Train] Loss: 1.9487, Accuracy: 0.1714 [Valid] Loss: 1.9494, Accuracy: 0.0580
[Epoch] 002  [Train] Loss: 1.9453, Accuracy: 0.1500 [Valid] Loss: 1.9476, Accuracy: 0.0580
[Epoch] 003  [Train] Loss: 1.9416, Accuracy: 0.1643 [Valid] Loss: 1.9449, Accuracy: 0.1040
[Epoch] 004  [Train] Loss: 1.9353, Accuracy: 0.1786 [Valid] Loss: 1.9419, Accuracy: 0.1220
[Epoch] 005  [Train] Loss: 1.9299, Accuracy: 0.2286 [Valid] Loss: 1.9388, Accuracy: 0.1220
[Epoch] 006  [Train] Loss: 1.9288, Accuracy: 0.1429 [Valid] Loss: 1.9359, Accuracy: 0.1280
[Epoch] 007  [Train] Loss: 1.9193, Accuracy: 0.1857 [Valid] Loss: 1.9325, Accuracy: 0.1460
[Epoch] 008  [Train] Loss: 1.9165, Accuracy: 0.2000 [Valid] Loss: 1.9294, Accuracy: 0.1880
[Epoch] 009  [Train] Loss: 1.9098, Accuracy: 0.2000 [Valid] Loss: 1.9259, Accuracy: 0.2360
[Epoch] 010  [Train] Loss: 1.9077, Accuracy: 0.2500 [Valid] Loss: 1.9224, Accuracy: 0.2580